In [ ]:
#default_exp speclib.library_base

In [ ]:
#export 
import pandas as pd

import alphabase.peptide.fragment as fragment

class SpecLibBase(object):
    def __init__(self,
        charged_ion_types:str, # e.g. ['b_1+','b_2+','y_1+','y_2+', ...]
        min_frag_mz = 100, max_frag_mz = 2000,
        min_precursor_mz = 400, max_precursor_mz = 6000,
    ):
        self.charged_ion_types = charged_ion_types
        self._precursor_df:pd.DataFrame = None
        self._fragment_inten_df:pd.DataFrame = None
        self._fragment_mass_df:pd.DataFrame = None
        self.min_frag_mz = min_frag_mz
        self.max_frag_mz = max_frag_mz
        self.min_precursor_mz = min_precursor_mz
        self.max_precursor_mz = max_precursor_mz

    @property
    def precursor_df(self):
        return self._precursor_df

    @precursor_df.setter
    def precursor_df(self, df):
        self._precursor_df = df.reset_index(drop=True)
        if 'precursor_mz' in self._precursor_df.columns:
            self.clip_precursor_()

    @property
    def fragment_mass_df(self):
        return self._fragment_mass_df

    @property
    def fragment_inten_df(self):
        return self._fragment_inten_df

    def clip_precursor_(self):
        ''' 
        Clip self._precursor_df inplace
        '''
        self._precursor_df = self._precursor_df[
            (self._precursor_df['precursor_mz']>=self.min_precursor_mz)&
            (self._precursor_df['precursor_mz']<=self.max_precursor_mz)
        ]
        self._precursor_df.reset_index(drop=True, inplace=True)

    def clip_inten_by_fragment_mass_(self):
        ''' 
        Clip self._fragment_inten_df inplace. All clipped masses are set as zeros.
        '''
        self._fragment_inten_df[
            (self._fragment_mass_df<self.min_frag_mz)|
            (self._fragment_mass_df>self.max_frag_mz)
        ] = 0

    def clip_inten_by_fragment_mass(self)->pd.DataFrame:
        df = self._fragment_inten_df.copy()
        df[
            (self._fragment_mass_df<self.min_frag_mz)|
            (self._fragment_mass_df>self.max_frag_mz)
        ] = 0
        return df
    
    def load_precursor_df(self, 
        precursor_files, **kwargs
    ):
        self._load_precursor_df(precursor_files, **kwargs)
        self.clip_precursor_()

    def _load_precursor_df(self, precursor_files, **kwargs):
        '''
        All sub-class must reimplement this method
        '''
        raise NotImplementedError(
            f'Sub-class of "{self.__class__}" must re-implement "_load_precursor_df()"'
        )

    def load_fragment_df(self, **kwargs):
        self.load_fragment_mass_df(**kwargs)
        self.load_fragment_inten_df(**kwargs)

    def load_fragment_inten_df(self, **kwargs):
        '''
        All sub-class must reimplement this method. 
        Fragment intensities can be predicted or from AlphaPept, or ...
        '''
        raise NotImplementedError(
            f'Sub-class of "{self.__class__}" must re-implement "load_fragment_inten_df()"'
        )

    def load_fragment_mass_df(self):
        (
            self._precursor_df, self._fragment_mass_df
        ) = fragment.get_fragment_mass_dataframe(
            self._precursor_df, self.charged_ion_types
        )
        # clip precursor after mass calculation
        self.clip_precursor_()

    def save_hdf(self, hdf_file):
        self._precursor_df.to_hdf(hdf_file, key='precursor', mode='w')
        self._fragment_mass_df.to_hdf(hdf_file, key='fragment_mass')
        self._fragment_inten_df.to_hdf(hdf_file, key='fragment_inten')

    def load_hdf(self, hdf_file):
        self._precursor_df = pd.read_hdf(hdf_file, key='precursor')
        self._fragment_mass_df = pd.read_hdf(hdf_file, key='fragment_mass')
        self._fragment_inten_df = pd.read_hdf(hdf_file, key='fragment_inten')
